In [3]:
# Install required packages
!pip install -q langchain_community langchain langchainhub tavily-python langchain-google-genai google-search-results requests


In [4]:
# Required Imports
import os
import datetime
import requests
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_community.utilities.tavily_search import TavilySearchAPIWrapper
from langchain import hub
from langchain.agents import AgentExecutor, create_openai_functions_agent
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.chains.conversation.memory import ConversationBufferMemory
from langchain.memory import ConversationSummaryBufferMemory
from langchain.prompts import PromptTemplate
from langchain.memory import ChatMessageHistory
from langchain.chains import ConversationChain
from kaggle_secrets import UserSecretsClient
from langchain.utilities import GoogleSerperAPIWrapper
from langchain import OpenAI, SerpAPIWrapper
from langchain.agents import initialize_agent, Tool,create_structured_chat_agent,create_react_agent
from langchain.agents import AgentType
from langchain.tools import StructuredTool
from langchain_core.runnables.history import RunnableWithMessageHistory
import json
from pydantic import BaseModel

In [16]:
# Initialize UserSecretsClient
user_secrets = UserSecretsClient()
gemini_api_key = user_secrets.get_secret("GEMINI")
# print(gemini_api_key)
gemini_api_key1 = user_secrets.get_secret("GEMINI1")
# print(gemini_api_key1)
TAVILY_API_KEY = user_secrets.get_secret("TAVILY_API_KEY")
secret_value_2 = user_secrets.get_secret("Serper")
# print(secret_value_2)
keyl = user_secrets.get_secret("LINKEDIN1")
# print(keyl)
os.environ["SERPER_API_KEY"] = secret_value_2


In [6]:
# Initialize the LLM for Google Generative AI (Gemini)
llm_general = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    api_key=gemini_api_key
)

llm_linkedin = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    temperature=0,
    api_key=gemini_api_key1
)


In [7]:
# Define the score-generating prompt template
classification_prompt = '''
You are a multipurpose chatbot with memory and you are given a user query. Assign a score out of 100 for each of the following parameters such that total sum must be 100 based on relevance:
1. User wants to check the LinkedIn profile of a person 
2. User wants to post ,edit or delete a post on LinkedIn.
3. User wants to check internet information about some person or even or if he mentions that he want realtime info or give any hint that he might need updatad data of any event.
4. User is prompting the chatbot for general conversation based on the past memory or general conversation or any humane emotion is in it.

Only provide four separate numbers separated by spaces, corresponding to the scores for each parameter in the order listed above. Do not include any additional text or explanations.
'''

# Task classifier function using the LLM
def classify_task(user_prompt, llm_classifier):
    prompt = classification_prompt + "\n\nUser: " + user_prompt + "\nYou:"
    response = llm_classifier.predict(prompt)
    print("Classification response:", response)
    try:
        scores = list(map(int, response.strip().split()))
        if len(scores) != 4:
            raise ValueError("Invalid number of scores returned.")
        return scores
    except Exception as e:
        print("Error parsing classification scores:", e)
        # Default to general conversation if classification fails
        return [0, 0, 0, 100]


In [8]:
common_memory =ConversationSummaryBufferMemory(llm=llm_general, max_token_limit=150)

/tmp/ipykernel_30/3885401533.py:1: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  common_memory =ConversationSummaryBufferMemory(llm=llm_general, max_token_limit=150)


In [9]:
def initialize_linkedin_search_agent():
    prompt = hub.pull("hwchase17/structured-chat-agent")
    search = GoogleSerperAPIWrapper()
    def get_organic_results(query):
        results = search.results(query)  
        json_data = json.dumps(results.get('organic', []) , indent=4)
        return json_data
    # Now define the tool using the function
    tools = [
        Tool(
            name="Internet Search",
            func=get_organic_results,
            description="Useful for extracting information and profiles that can be realtime from Google  .",
        )

    ]
    agent = create_structured_chat_agent(llm_linkedin, 
                                    tools, 
                                    prompt)

    # Set up an executor for the agent, specifying the agent, tools, and enabling verbose output
    agent_executor = AgentExecutor(agent=agent, 
                                   tools=tools,
                                   verbose=True,memory=common_memory)
    

    return agent_executor


In [10]:
class UpdatePostInput(BaseModel):
    post_id: str
    new_text: str

def linkedin_post_agent(text, media=False):
    key = keyl  # Insert your LinkedIn API key here
    get_header = {
        'Authorization': f'Bearer {key}'
    }

    r = requests.get('https://api.linkedin.com/v2/userinfo', headers=get_header)
    di = r.json()
    user_id = di['sub']

    body_text = {
        "author": f"urn:li:person:{user_id}",
        "lifecycleState": "PUBLISHED",
        "specificContent": {
            "com.linkedin.ugc.ShareContent": {
                "shareCommentary": {
                    "text": text  # Use the provided text
                },
                "shareMediaCategory": "NONE" if not media else "IMAGE"  # Handle media parameter
            }
        },
        "visibility": {
            "com.linkedin.ugc.MemberNetworkVisibility": "PUBLIC"
        }
    }

    post_text = requests.post('https://api.linkedin.com/v2/ugcPosts', json=body_text, headers=get_header)
    return post_text.json()


def delete_linkedin_share(share_id: str) -> str:
    key = keyl  # Insert your LinkedIn API key here
    headers = {
        'Authorization': f'Bearer {key}',
        'X-Restli-Protocol-Version': '2.0.0',
        'Content-Type': 'application/json'
    }

    # Send the DELETE request to remove the post
    response = requests.delete(f'https://api.linkedin.com/v2/shares/{share_id}', headers=headers)

    if response.status_code == 200:
        return "Share deleted successfully."
    else:
        return f"Failed to delete share. Status Code: {response.status_code}, Response: {response.text}"


def update_linkedin(post_id: str, new_text: str):
    # First delete the old post
    delete_response = delete_linkedin_share(post_id)
    if "successfully" in delete_response:
        # Create the new post with updated text
#         print("haha")
        return linkedin_post_agent(new_text)
    else:
        return delete_response


# Define the LinkedIn agent setup
def initialize_linkedin_post_agent():
    # Pydantic model for input validation

    # Define tools for the agent
    prompt = hub.pull("hwchase17/structured-chat-agent")
    tools = [
        Tool(
            name="Create LinkedIn Post",
            func=lambda text: linkedin_post_agent(text),
            description="Use this tool to create a LinkedIn post with optional media. It takes a string as input."
        ),
        Tool(
            name="Delete LinkedIn Post",
            func=lambda share_id: delete_linkedin_share(share_id),
            description="Use this tool to delete an existing LinkedIn post by providing the post ID."
        ),
        StructuredTool(
            name="Edit LinkedIn Post",
            func=lambda post_id, new_text: update_linkedin(post_id, new_text),
            description="Use this tool to update an existing LinkedIn post by providing the post ID and new text.",
            args_schema=UpdatePostInput   # Structured input for post_id and new_text
        )
    ]

    agent = create_structured_chat_agent(llm_general, 
                                    tools, 
                                    prompt)

    # Set up an executor for the agent, specifying the agent, tools, and enabling verbose output
    agent_executor = AgentExecutor(agent=agent, 
                                   tools=tools,
                                   verbose=True,memory=common_memory)
    

    return agent_executor


In [11]:
def initialize_general_chatbot():
#     memory = ConversationBufferMemory()
    conversation = ConversationChain(llm=llm_linkedin, verbose=True, memory=common_memory)
    return conversation


In [19]:
def execute_linkedin_search(agent_executor, user_query):
    # Add the current date to the query
    current_date = datetime.datetime.now().strftime('%Y-%m-%d')
#     query = f"{user_query} {current_date} ?"
    query = f"{user_query}  ?"
    
    # Execute the query with the agent
    result = agent_executor.invoke({"input": query})
#     print("LinkedIn Search Result 1:", result['output'])
    print(result['output'])
    return result

def execute_linkedin_post(agent_executor, user_query):
    # Add the current date to the query
    current_date = datetime.datetime.now().strftime('%Y-%m-%d')
    query = f"{user_query} {current_date}"
    
    # Execute the query with the agent
    result = agent_executor.invoke({"input": query})
#     print("LinkedIn Post Result:", result['output'])
    print(result['output'])
    return result

def execute_general_chat(conversation, user_query):
    response = conversation.predict(input=user_query)
#     print(f"Chatbot: {response}")
    print(f"{response}")
    return response


In [13]:
def main():
    # Initialize agents
    linkedin_search_agent = initialize_linkedin_search_agent()
    linkedin_post_agent = initialize_linkedin_post_agent()
    general_conversation = initialize_general_chatbot()
    
    print("Chatbot: Hello! You can talk to me. Type 'exit' or 'quit' to stop the conversation.")
    
    
    while True:
        user_input = input("You: ")
        if user_input.lower() in ["exit", "quit"]:
            print("Chatbot: Goodbye!")
            break
        
        # Classify the task for the current user input
        scores = classify_task(user_input, llm_general)
#         print("Classification scores:", scores)
        
        # Determine the highest scoring task
        max_score = max(scores)
        task_index = scores.index(max_score)
        
        # Map index to task
        task_mapping = {
            0: "LinkedIn Profile Search",
            1: "LinkedIn Post",
            2: "Internet Information Search",
            3: "General Conversation"
        }
        selected_task = task_mapping.get(task_index, "General Conversation")
#         print(f"Detected Task: {selected_task}")
        
        # Route the user input to the appropriate agent
        if task_index == 0:
            print("Task: LinkedIn Profile Search")
            execute_linkedin_search(linkedin_search_agent, user_input)
        elif task_index == 1:
            print("Task: LinkedIn Post")
            execute_linkedin_post(linkedin_post_agent, user_input)
        elif task_index == 2:
            print("Task: Internet Information Search")
            execute_linkedin_search(linkedin_search_agent, user_input)  # Reuse search agent or create a separate one
        else:
            print("Task: General Conversation")
            execute_general_chat(general_conversation, user_input)
        
        # Optionally, you can add a confirmation step or allow explicit task switching


In [14]:
main()

/opt/conda/lib/python3.10/site-packages/langsmith/client.py:354: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/langsmith/client.py:354: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(
/tmp/ipykernel_30/3281030631.py:3: LangChainDeprecationWarning: The class `ConversationChain` was deprecated in LangChain 0.2.7 and will be removed in 1.0. Use :meth:`~RunnableWithMessageHistory: https://python.langchain.com/v0.2/api_reference/core/runnables/langchain_core.runnables.history.RunnableWithMessageHistory.html` instead.
  conversation = ConversationChain(llm=llm_linkedin, verbose=True, memory=common_memory)


Chatbot: Hello! You can talk to me. Type 'exit' or 'quit' to stop the conversation.


You:  hey how are you?


/tmp/ipykernel_30/3069878565.py:15: LangChainDeprecationWarning: The method `BaseChatModel.predict` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = llm_classifier.predict(prompt)


Classification response: 0 0 0 100 

Task: General Conversation


> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: hey how are you?
AI:

> Finished chain.
Hey there! I'm doing well, thanks for asking. I'm always learning and processing information, so I'm feeling pretty energized. How about you? What's going on in your world today? 



...

Classification response: 0 0 0 100 

Task: General Conversation


> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
System: The AI is doing well and is always learning and processing information. It asks the human how they are doing. The human introduces themselves as Harshit and asks the AI to address them as such. The AI agrees and asks Harshit what's on their mind. Harshit asks for a joke, and the AI tells them a joke about atoms. Harshit then tells the AI that they are working on a project to create a multimodal chatbot. The AI is excited to hear about this and asks Harshit what modalities they are planning to include. The AI is curious about the project and asks Harshit about their goals and challenges. Harshit men

You:  hey you are awesome bye bye thank you


Classification response: 0 0 0 100 

Task: General Conversation


> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
System: The AI is doing well and is always learning and processing information. It asks the human how they are doing. The human introduces themselves as Harshit and asks the AI to address them as such. The AI agrees and asks Harshit what's on their mind. Harshit asks for a joke, and the AI tells them a joke about atoms. Harshit then tells the AI that they are working on a project to create a multimodal chatbot. The AI is excited to hear about this and asks Harshit what modalities they are planning to include. The AI is curious about the project and asks Harshit about their goals and challenges. Harshit men

You:  exit


Chatbot: Goodbye!


In [21]:
main()

/opt/conda/lib/python3.10/site-packages/langsmith/client.py:354: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/langsmith/client.py:354: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


Chatbot: Hello! You can talk to me. Type 'exit' or 'quit' to stop the conversation.


You:  write a post of harms of AI


Classification response: 0 100 0 0 

Task: LinkedIn Post


> Entering new AgentExecutor chain...
Thought: I need to create a LinkedIn post about the harms of AI. I'll use string formatting to make it more engaging.
Action:
```json
{
  "action": "Create LinkedIn Post",
  "action_input": "## The Dark Side of AI: What We Need to Talk About in 2024\n\nAI is rapidly changing our world, but it's not all sunshine and rainbows. As we embrace the potential of AI, we must also acknowledge its potential harms. Here are some key concerns:\n\n* **Job displacement:** AI automation is already impacting certain industries, and this trend is likely to accelerate. We need to prepare for a future where many jobs are automated and focus on reskilling and upskilling the workforce.\n* **Bias and discrimination:** AI systems can perpetuate and amplify existing societal biases if they are trained on biased data. This can lead to unfair outcomes for certain groups of people.\n* **Privacy and surveillance:** AI

KeyboardInterrupt: Interrupted by user

## the function with linkedin and web scrapping both together as a tool , we were not able to implement it because of very limited tokens by proxycurl

In [62]:
import json
import requests

def web_search_and_linkedin_scrape(query):
    
    results = search.results(query)
    organic_results = results.get('organic', [])
    
    # Step 2: Format the search results to match the required structure
    formatted_results = []
    for idx, result in enumerate(organic_results):
        formatted_result = {
            "title": result.get('title', ''),
            "link": result.get('link', ''),
            "snippet": result.get('snippet', ''),
            "attributes": {
                "Missing": result.get('missing', 'None')
            },
            "position": idx + 1
        }
        formatted_results.append(formatted_result)
    
    linkedin_urls = [result['link'] for result in formatted_results if 'linkedin.com/in/' in result['link']]
    
    profile_data = None
   
    if linkedin_urls:
        first_linkedin_url = linkedin_urls[0]  
        profile_data = scrape_linkedin_profile(first_linkedin_url)  

        linkedin_result = {
            "title": f"LinkedIn Profile: {profile_data.get('full_name', 'Profile')}",
            "link": first_linkedin_url,
            "snippet": f"{profile_data.get('headline', 'No headline available')}",
            "attributes": {
                "profile": profile_data
            },
            "position": len(formatted_results) + 1  
        }
        
        formatted_results.append(linkedin_result)
    json_data = json.dumps(formatted_results, indent=1)

    return json_data

def scrape_linkedin_profile(linkedin_profile_url:str):
    """Scrape information from LinkedIn profiles and remove links to reduce token usage."""
    api_endpoint = 'https://nubela.co/proxycurl/api/v2/linkedin'
    headers = {"Authorization": f'Bearer {"eUofCbySh2p2unbHTnv8cg"}'}

    response = requests.get(
        api_endpoint, params={"url": linkedin_profile_url, "skills": "include"}, headers=headers
    )

    data = response.json()
    cleaned_data = {
        k: v
        for k, v in data.items()
        if v and k not in ["people_also_viewed", "certifications", "profile_pic_url", "linkedin_url"]
    }
    
    # Remove links inside 'groups' if present
    if 'groups' in cleaned_data:
        for group_dict in cleaned_data['groups']:
            group_dict.pop('profile_pic_url', None)
            group_dict.pop('linkedin_url', None)

    return cleaned_data


In [63]:
prompt = hub.pull("hwchase17/structured-chat-agent")
search = GoogleSerperAPIWrapper()

hi= web_search_and_linkedin_scrape("trijal srivastava iitk")

/opt/conda/lib/python3.10/site-packages/langsmith/client.py:354: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [64]:
print(hi)

[
 {
  "title": "Trijal Srivastava - Leader - IITK Consulting Group | LinkedIn",
  "link": "https://in.linkedin.com/in/trijal-srivastava-944056228",
  "snippet": "I am Trijal Srivastava, a Third-year undergraduate in the Department of Electrical Engineering. I am passionate about the growing infrastructure and ...",
  "attributes": {
   "Missing": "None"
  },
  "position": 1
 },
 {
  "title": "2 \"Trijal Srivastava\" profiles - LinkedIn",
  "link": "https://www.linkedin.com/pub/dir/Trijal/Srivastava",
  "snippet": "Trijal Srivastava. Incoming AMTS intern @Salesforce | Leader @IITK Consulting Group | Junior undergraduate, IIT Kanpur | AI for social good.",
  "attributes": {
   "Missing": "None"
  },
  "position": 2
 },
 {
  "title": "Trijal Srivastava (@trijal_xo) \u2022 Instagram profile",
  "link": "https://www.instagram.com/trijal_xo/",
  "snippet": "Happiness exists, when you don't know a thing. IITK'26. 0 posts. 792 followers. 868 following. This account is private. Already follow 